In [1]:
import pandas as pd
from transformers import AutoTokenizer
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from utils import emoji_dict, stop_words
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
import numpy as np


d:\COURSE\Computer science\Bangkit\Project\GradioProto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = load_model('model/bert_attention.h5')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 128, 768)  │ 38,400,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128, 256)  │    918,528 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 128, 256)  │          0 │ bidirectional[0]… │
│ (Attention)         │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        387 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,351,814 (150.12 MB)

 Trainable params: 951,811 (3.63 MB)

 Non-trainable params: 38,400,000 (146.48 MB)

 Optimizer params: 3 (16.00 B)

In [3]:
url = 'https://drive.google.com/file/d/1O6Nyg8yI4Kb-VaOlZr33_zkDGBDMrfwg/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

train_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1EeIIXUOEQDaHvTDGGvpa61BNVe-ggJIV/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

valid_df = pd.read_csv(url)

url = 'https://drive.google.com/file/d/1Yh4CMGWT-ERZ6T7Q7JPVf2ZjGZx9Sh_O/view?usp=drive_link'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]

test_df = pd.read_csv(url)

In [4]:
train_df['reviews'] = train_df['reviews'].astype(str)
valid_df['reviews'] = valid_df['reviews'].astype(str)
test_df['reviews'] = test_df['reviews'].astype(str)

In [5]:
url = 'https://drive.google.com/file/d/1-wpLUWpIVBTr0BLHG79cZhHrZ34KFOJs/view?usp=drive_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
data_df = pd.read_csv(url)
data_df['reviews'] = data_df['reviews'].astype(str)

In [6]:
data_df.columns = train_df.columns

train_data_df, valid_data_df = train_test_split(data_df, train_size=4000, random_state=42)

train_df = pd.concat([train_df, train_data_df], ignore_index=True)
valid_df = pd.concat([valid_df, valid_data_df], ignore_index=True)

print(f"Length of training data: {len(train_df)}")
print(f"Length of validation data: {len(valid_df)}")
print(f"Length of test data: {len(test_df)}")

Length of training data: 13140
Length of validation data: 1402
Length of test data: 463


In [7]:
train_df

,reviews,negative,neutral,positive
0,warung dimiliki pengusaha pabrik puluhan terke...,False,False,True
1,lokasi strategis jalan sumatera bandung nya ny...,False,False,True
2,betapa bahagia nya unboxing paket barang nya b...,False,False,True
3,makanan beragam harga makanan food stall kasir...,False,False,True
4,pakai kartu kredit bca tidak untung rugi besar,True,False,False
...,...,...,...,...
13135,masker camille emang bagus woii lahh emang mas...,False,False,True
13136,ready kah,False,True,False
13137,emang langsung putih tpi jgn coba kulit badak ...,False,False,True
13138,orang jateng cocok bgt manissss,False,False,True


In [8]:
# kamus_alay_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')
# kamus_alay_df

In [9]:
# kamus_alay_dict = kamus_alay_df.set_index('slang')['formal'].to_dict()

In [10]:
# def df_to_dataset(df, shuffle=True, batch_size=32):
#     """
#     Converts a pandas DataFrame into a TensorFlow Dataset.
#     - `reviews` is the text input.
#     - Target columns are `negative`, `neutral`, and `positive`.
#     """
#     features = df['reviews']  # Input text
#     labels = df[['negative', 'neutral', 'positive']]  # Multi-class targets
#     ds = tf.data.Dataset.from_tensor_slices((features, labels))
#     if shuffle:
#         ds = ds.shuffle(buffer_size=len(df))
#     ds = ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return ds

# # Convert the datasets
# batch_size = 32
# train_ds = df_to_dataset(train_df, shuffle=True, batch_size=batch_size)
# valid_ds = df_to_dataset(valid_df, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test_df, shuffle=False, batch_size=batch_size)

In [11]:
# def preprocess(reviews, label):
#     """
#     Preprocess the reviews using kamus_alay_dict.
#     Replace words if found in the dictionary; otherwise, keep the original word.
#     """
#     # Split the reviews into words
#     words = tf.strings.split(reviews)

#     # Map the dictionary lookup to each word
#     def replace_word(word):
#         # Use a Python dictionary lookup wrapped in tf.py_function
#         def lookup(w):
#             return kamus_alay_dict.get(w, w)  # Replace with dict value or keep the same
        
#         # Apply the lookup function and return
#         return tf.py_function(func=lookup, inp=[word], Tout=tf.string)

#     # Replace each word using map_fn
#     words = tf.map_fn(replace_word, words, fn_output_signature=tf.string)

#     # Join words back into a string
#     processed_review = tf.strings.reduce_join(words, separator=" ")

#     return processed_review, label



# train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
# valid_ds = valid_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
# test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [13]:
# import tensorflow as tf

# def tokenize(text):
#     # Tokenize the text using the tokenizer
#     return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")['input_ids']

# # Apply tokenization to each review
# X_train_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in train_df['reviews']],
#     dtype=tf.int32
# )

# X_valid_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in valid_df['reviews']],
#     dtype=tf.int32
# )

# X_test_tokenized = tf.convert_to_tensor(
#     [tokenize(text) for text in test_df['reviews']],
#     dtype=tf.int32
# )

# # Convert labels to tensors
# y_train = tf.convert_to_tensor(train_df.iloc[:, 1:].values, dtype=tf.int32)
# y_valid = tf.convert_to_tensor(valid_df.iloc[:, 1:].values, dtype=tf.int32)
# y_test = tf.convert_to_tensor(test_df.iloc[:, 1:].values, dtype=tf.int32)

# X_train_tokenized = tf.reshape(X_train_tokenized, (-1, 128))
# X_valid_tokenized = tf.reshape(X_valid_tokenized, (-1, 128))
# X_test_tokenized = tf.reshape(X_test_tokenized, (-1, 128))

# # Check the shapes of the tokenized datasets

# print(X_train_tokenized.shape)
# print(X_valid_tokenized.shape)
# print(X_test_tokenized.shape)

def tokenize(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")['input_ids']

X_train_tokenized = tf.squeeze(tf.stack(train_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)
X_valid_tokenized = tf.squeeze(tf.stack(valid_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)
X_test_tokenized = tf.squeeze(tf.stack(test_df['reviews'].map(lambda x: tokenize(x)).tolist()), axis=1)

y_train = tf.convert_to_tensor(train_df.iloc[1:, 1:].values)
y_valid = tf.convert_to_tensor(valid_df.iloc[:, 1:].values)
y_test = tf.convert_to_tensor(test_df.iloc[:, 1:].values)

In [14]:
predictions = model.predict(X_test_tokenized)
predictions

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step


array([[ 2.9563203 , -8.189568  , -0.4949314 ],
       [ 3.7966728 , -9.818038  , -0.82628053],
       [ 6.084931  , -8.723497  , -4.0352902 ],
       ...,
       [-2.646826  ,  0.17683819,  1.4340731 ],
       [-2.3264742 ,  0.52248657,  1.1510234 ],
       [ 0.12442549, -1.3774743 ,  0.16445436]], dtype=float32)

In [27]:
import heapq

def get_top_3_positive_index(predictions):
    positive_preds = predictions[:, 2]
    pq = []
    for i, pred in enumerate(positive_preds):
        if np.argmax(predictions[i]) == 2:
            heapq.heappush(pq, (-pred, i))
    top_3 = []
    for _ in range(3):
        if pq:
            _, task = heapq.heappop(pq)
            top_3.append(task)
    return top_3

def get_top_3_negative_index(predictions):
    negative_preds = predictions[:, 0]
    pq = []
    for i, pred in enumerate(negative_preds):
        if np.argmax(predictions[i]) == 0:
            heapq.heappush(pq, (-pred, i))
    top_3 = []
    for _ in range(3):
        if pq:
            _, task = heapq.heappop(pq)
            top_3.append(task)
    return top_3


In [28]:
def indices_to_texts(indices, data):
    texts = []
    for index in indices:
        texts.append(data[index])
    return texts

def get_top_3_positive_comments(predictions, data):
    top_3_positive_idx = get_top_3_positive_index(predictions)
    comments = indices_to_texts(top_3_positive_idx, data)

    return comments

def get_top_3_negative_comments(predictions, data):
    top_3_negative_idx = get_top_3_negative_index(predictions)
    comments = indices_to_texts(top_3_negative_idx, data)

    return comments

In [34]:
valid_predictions = model.predict(X_valid_tokenized)
print(get_top_3_negative_comments(valid_predictions, valid_df['reviews']))
print(get_top_3_positive_comments(valid_predictions, valid_df['reviews']))

44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step
['kecewa banget net direkam tidak firstwkan musim show bener nya diomongin knetz net tidak sesukses sunbae nya tolong czennies kerja keras gue tidak sanggup net knets grup gagal nya', 'marah woy tidak tl tidak scroll tidak refresh mental mengirim pesan langsung lama', 'perlakuan tidak mengenakkan hotel geram marah pelayanan saking buruk nya tidak mengerti hotel berani mencuri barang tidak orang tinggal hotel']
['bandung lewatkan ngafe valey dago pakar menikmati makan malam menikmati pemandangan bagus makanan nya enak coba menginap hotel nya berbentuk caravan asyik banget udara nya segar sarapan nya mantap linting capgomeh nya enak banget', 'suasana resto cukup nyaman layanan cukup baik menawarkan beragam menu bahan dasar seafood udang gurame bawal kakap kepiting ayam sayuran gurame goreng gurih renyah sambal mangga nya enak nikmat ayam goreng fillet nya gurih enak gurame bakar kecap dibakar sempurna gurih ratarata masakan nya enak', 'sesuai nama 

In [31]:
get_top_3_negative_comments(predictions, test_df['reviews'])

['kecewa aplikasi hubungi pusat panggilan cek pemesanan bayar tidak muncul e tiket nya kirim bukti transaksi nya email tidak pusat panggilan menghubungi kecewa',
 'mengecewakan pesan tiket dibayar dikasih bukti transfer langsung dikasih nomor e tiket nya tidak konfirmasi mendekati h pesanan kadaluarsa pelayanan buruk pelayanan pelanggan nya tidak sopan kembalikan uang pengembalian dana tiket',
 'menyesal tidak sih ganti handphone iphone samsung s hmm menyesal kurang enak pakai s segi camera suka ip segi performa ip batre tidak beda jauh banget s melar buka instagram ckck']

In [32]:
get_top_3_positive_comments(predictions, test_df['reviews'])

['hotel salah hotel wajib dikunjungi buka puasa bareng nya menarik mewah staf nya ramah makanan nya banyaakk jenis nya menu nya komplet suasana nya menyenangkan indah',
 'kuliner dicoba ketan jompo gurih manis hangat nya ketan ditaburi bubuk koyah banget menemani malam kota jember harga nya murah ribu suka banget',
 'menginap malam keluarga hotel nya bagus bersih fasilitas hotel lengkap breakfast nya enakenak suasana hotel sejuk nyaman cocok pilihan menginap keluarga']

In [56]:
test_df['reviews'][22]

'kecewa aplikasi hubungi pusat panggilan cek pemesanan bayar tidak muncul e tiket nya kirim bukti transaksi nya email tidak pusat panggilan menghubungi kecewa'

In [15]:
def get_prediction_class(predictions):
    m, n = predictions.shape
    y_pred_class = np.zeros((m, n))
    for i in range(m):
        max_val = 1e-3
        max_j = -1
        for j in range(n):
            if max_val < predictions[i, j]:
                max_val = predictions[i, j]
                max_j = j
        y_pred_class[i, max_j] = 1
    y_pred_class = y_pred_class.astype(bool)
    y_pred_class = np.argwhere(y_pred_class)[:, 1]
    return y_pred_class

In [16]:
predictions_class = get_prediction_class(predictions)
predictions_class

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2,
       2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 1, 1, 2, 0, 1, 0,
       1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,

In [17]:
max_pred = np.max(predictions, axis=1)
max_pred

array([ 2.9563203 ,  3.7966728 ,  6.084931  ,  3.2185571 ,  5.9945383 ,
        4.831225  ,  9.618999  , 10.481339  ,  7.7947187 , 10.758614  ,
        2.8468685 ,  7.9409823 ,  7.125659  ,  8.052052  ,  3.7959647 ,
        5.5193553 ,  2.7210753 ,  9.352341  ,  9.726931  ,  1.1093416 ,
        7.319113  ,  9.155948  , 13.418212  ,  7.4314623 ,  5.301455  ,
        5.6957545 ,  5.8813934 ,  5.333892  , 13.114037  , 10.289615  ,
        6.4975467 ,  3.5062249 ,  6.251109  ,  3.3340983 ,  5.06653   ,
        7.6433773 ,  9.727084  ,  5.092947  ,  7.302062  ,  3.4700015 ,
        8.18744   ,  6.679376  ,  9.967164  ,  8.026849  ,  0.95971805,
        6.480126  ,  6.552089  ,  7.834458  ,  4.5079656 ,  1.2110353 ,
        6.996067  ,  6.041851  ,  7.955485  ,  3.3749886 ,  9.0804    ,
        5.4823003 ,  3.6090188 ,  0.29456648,  6.7301154 ,  4.945892  ,
        0.78090763,  3.333691  ,  9.411411  ,  8.45279   ,  6.165317  ,
        5.3108025 ,  5.735136  ,  6.6922646 , 10.96574   ,  3.06